In [1]:
!rm -fr r_trader out
!mkdir out
!git clone https://github.com/abreham-atlaw/r_trader
!cd r_trader && git checkout deep-reinforcement.it-42.training
!pip install cattrs positional-encodings==6.0.1 dropbox pymongo==4.3.3 dependency-injector==4.41.0

Cloning into 'r_trader'...
remote: Enumerating objects: 25004, done.
remote: Counting objects: 100% (616/616), done.
remote: Compressing objects: 100% (340/340), done.
remote: Total 25004 (delta 393), reused 446 (delta 270), pack-reused 24388 (from 1)
Receiving objects: 100% (25004/25004), 78.61 MiB | 26.03 MiB/s, done.
Resolving deltas: 100% (18462/18462), done.
Branch 'deep-reinforcement.it-42.training' set up to track remote branch 'deep-reinforcement.it-42.training' from 'origin'.
Switched to a new branch 'deep-reinforcement.it-42.training'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.9/494.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 71.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.1/572.1 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: six
    Found existing

In [2]:
import os
KAGGLE_ENV = os.path.exists("/kaggle/working")
REPO_PATH = "/kaggle/working/r_trader" if KAGGLE_ENV else "/content/r_trader"

print(f"KAGGLE ENV: {KAGGLE_ENV}")

import sys
sys.path.append(REPO_PATH)

In [3]:
import os

from lib.utils.torch_utils.model_handler import ModelHandler
from core.di import ServiceProvider
from lib.utils.logger import Logger
from core import Config

from core.utils.research.model.layers import L1Norm
from core.utils.research.utils.ensemble.pg_msm.pg_msm_builder import PerformanceGridMSMBuilder
from core.utils.research.losses import ProximalMaskedLoss, MeanSquaredErrorLoss, MultiLoss, ScoreLoss, SoftConfidenceScore
from core.utils.research.data.prepare.swg.manipulate import ReciprocateSampleWeightManipulator
from core.utils.research.data.prepare.swg.swg_pipeline import SampleWeightGeneratorPipeline
from core.utils.research.data.prepare.swg.xswg import MomentumXSampleWeightGenerator
from core.utils.research.utils.model_evaluator import ModelEvaluator
from core.utils.kaggle.data_repository import KaggleDataRepository

 PID:35 [2025-07-13 07:03:56.709311]  XLA is not installed. Training using TPU will not be possible. 


In [ ]:
def download_data(root, datasets, zip_filename, kernel_mode=True, checksums=None):
    repository = KaggleDataRepository(
        output_path=root,
        zip_filename=zip_filename
    )
    repository.download_multiple(datasets, kernel=kernel_mode, checksums=checksums)
    os.system(f"unzip -d root/")

In [4]:
DATA_ROOT = "/kaggle/input" if KAGGLE_ENV else "/content/input"

DATASETS = [
    f"abrehamatlaw0/spinoza-ds-datapreparer-simsim-it-42-{i}"
    for i in range(4)
]
CHECKSUMS = [
    'a42f2e67483d937fa57026d3af3c95504caf3594872121059af04d0bd079a3a9',
]
CHECKSUMS = None
KERNEL_MODE = False
ZIP_FILENAME = "out.zip"
if not KAGGLE_ENV:
    download_data(DATA_ROOT, DATASETS, ZIP_FILENAME, kernel_mode=KERNEL_MODE, checksums=CHECKSUMS)

CONTAINERS = [os.path.join(DATA_ROOT, container) for container in os.listdir(DATA_ROOT)]
DATA_PATHES, TEST_DATA_PATHES = [
    [
        os.path.join(container, "out", type_)
        for container in CONTAINERS
    ]
    for type_ in ["train", "test"]
]

MODEL_ID = "abrehamalemu-spinoza-training-ensemble-pgmsm-0-it-42"

TMP_PATH = "/kaggle/working/out/"
SAVE_PATH = os.path.join(TMP_PATH, f"{MODEL_ID}.zip")
UPLOAD_PATH = "/Apps/RTrader/"

MODELS_PATHS = [
    f"/Apps/RTrader/{model_id}.zip"
    for model_id in [
        f"abrehamalemu-spinoza-training-pgmsm-cnn-{i}-it-42-tot"
        for i in [0, 1]
    ]
]

WEIGHT_GENERATORS = [
    MomentumXSampleWeightGenerator(),
    SampleWeightGeneratorPipeline(
        generators=[
            MomentumXSampleWeightGenerator(),
            ReciprocateSampleWeightManipulator(),
        ]
    )
]

In [5]:
fs = ServiceProvider.provide_file_storage()

In [6]:
def load_models():
    
    def load_model(model_path):
        Logger.info(f"Loading {model_path}...")
        local_path = os.path.basename(model_path)
        if not os.path.exists(local_path):
            fs.download(model_path, local_path)
        return ModelHandler.load(local_path)

    fs = ServiceProvider.provide_file_storage()
    
    return [
        load_model(path)
        for path in MODELS_PATHS
    ]

In [7]:
models = load_models()

 PID:35 [2025-07-13 07:03:56.819106]  Loading /Apps/RTrader/abrehamalemu-spinoza-training-pgmsm-cnn-0-it-42-tot.zip... 
 PID:35 [2025-07-13 07:03:59.541106]  Using Storage 2 for /Apps/RTrader/abrehamalemu-spinoza-training-pgmsm-cnn-0-it-42-tot.zip 


2025-07-13 07:04:00 URL:http://p-lux3.pcloud.com/cBZAuaFccZjfeSBD7Znvzs7ZZrAeMVkZqZZGhkZkZOQuiZ9LZezZPpZp8ZzzZFLZKFZGQZ0zZ6RZCRZtpZUHZGzZS3JB5Z7Rw68XbR96HwLRzwNW0cbpj6PiB7/abrehamalemu-spinoza-training-pgmsm-cnn-0-it-42-tot.zip [13024524/13024524] -> "abrehamalemu-spinoza-training-pgmsm-cnn-0-it-42-tot.zip" [1]


 PID:35 [2025-07-13 07:04:00.664224]  [CNN2] Building... 
 PID:35 [2025-07-13 07:04:00.673603]  [EmbeddingBlock] Building... 
 PID:35 [2025-07-13 07:04:00.677543]  [Indicators] Building... 
 PID:35 [2025-07-13 07:04:00.690742]  [CNNBlock] Building... 
 PID:35 [2025-07-13 07:04:00.909950]  [CollapseBlock] Building... 
 PID:35 [2025-07-13 07:04:00.911726]  [LinearModel] Building... 
 PID:35 [2025-07-13 07:04:00.983034]  Loading /Apps/RTrader/abrehamalemu-spinoza-training-pgmsm-cnn-1-it-42-tot.zip... 
 PID:35 [2025-07-13 07:04:02.483271]  Using Storage 1 for /Apps/RTrader/abrehamalemu-spinoza-training-pgmsm-cnn-1-it-42-tot.zip 
 PID:35 [2025-07-13 07:04:03.507023]  [CNN2] Building... 
 PID:35 [2025-07-13 07:04:03.507268]  [EmbeddingBlock] Building... 
 PID:35 [2025-07-13 07:04:03.507710]  [Indicators] Building... 
 PID:35 [2025-07-13 07:04:03.508017]  [CNNBlock] Building... 
 PID:35 [2025-07-13 07:04:03.525404]  [CollapseBlock] Building... 
 PID:35 [2025-07-13 07:04:03.525865]  [LinearMod

2025-07-13 07:04:03 URL:http://p-lux1.pcloud.com/cBZaUVpccZv5fSBD7ZHDks7ZZyNeMVkZqZZGhkZkZOQuiZ3FZ0QZr4ZnRZm4ZhQZWzZXzZcpZhRZzYZU4Z9HZnYZs3JB5ZyFXCM1iFxvQ5CkqbudOGlJqgXpWV/abrehamalemu-spinoza-training-pgmsm-cnn-1-it-42-tot.zip [13024524/13024524] -> "abrehamalemu-spinoza-training-pgmsm-cnn-1-it-42-tot.zip" [1]


In [8]:
loss = ProximalMaskedLoss(
    n=len(Config.AGENT_STATE_CHANGE_DELTA_STATIC_BOUND) + 1 ,
    p=1,
    softmax=True,
    collapsed=True
)

In [9]:
builder = PerformanceGridMSMBuilder(
    data_paths=DATA_PATHES,
    generators=WEIGHT_GENERATORS,
    tmp_path=TMP_PATH,
    loss=loss
)

 PID:35 [2025-07-13 07:04:03.609241]  Detected that loss is either not SpinozaLoss or Sample Weight is not enabled. 


In [10]:
evaluator = ModelEvaluator(
    cls_loss_fn=loss,
    data_path=TEST_DATA_PATHES,
)

In [ ]:
model = builder.build(
    models=models,
    activation=L1Norm()
)

 PID:35 [2025-07-13 07:04:03.648756]  Generating weights... 
 PID:35 [2025-07-13 07:04:03.648953]  Generating weights for Generator: MomentumXSampleWeightGenerator... 
 PID:35 [2025-07-13 07:04:03.649360]  Setting up... 
 PID:35 [2025-07-13 07:04:03.649462]  Processing 4 directories... 
 PID:35 [2025-07-13 07:04:03.649495]  Processing ('/kaggle/input/rtrader-datapreparer-simsim-it-42-3/out/train/X',) 
 PID:35 [2025-07-13 07:07:11.212726]  Successfully processed files! 
 PID:35 [2025-07-13 07:07:11.213215]  Processing ('/kaggle/input/rtrader-datapreparer-simsim-it-42-1/out/train/X',) 


In [ ]:
ModelHandler.save(model, SAVE_PATH)
fs.upload_file(SAVE_PATH, UPLOAD_PATH)

In [ ]:
evaluator.evaluate(model)

In [ ]:
!rm -fr r_trader